# Code for basin delimitation

Developed by Rodrigo Aguayo (2020-2023)

In [ ]:
# grass
import grass.jupyter as gj
import grass.script as gs
from grass.pygrass.modules.grid import GridModule

# spatial
from geocube.vector import vectorize
import rioxarray as rioxr
import geopandas as gpd
import regionmask

# basic
import pandas as pd
import numpy as np
import os

os.chdir('/home/rooda/Dropbox/Patagonia') 

## Elevation

In [ ]:
# import raster NASADEM 30 m
gs.run_command("r.import", input="GIS South/dem_patagonia3f_subset.tif", output="elevation_lr", flags = "o", overwrite=True)
gs.run_command("g.region", raster="elevation_lr", flags="p")

In [ ]:
# sink removal using Lindsay et al. (2005) [MULTICORE]
grid = GridModule("r.hydrodem", input="elevation_lr", 
                  output = "elevation_filled",
                  overwrite = True, processes = 20)
grid.run()

## Delimitate all basins 

In [ ]:
# calculate accumulation raster map and drainage direction raster map using MFD: multiple flow direction
gs.run_command("r.stream.extract", 
                  elevation = "elevation_filled",  
                  threshold = 200,
                  direction = "fdir", 
                  stream_raster = "stream_r",
                  overwrite = True)

In [ ]:
gs.run_command("r.stream.basins", 
               direction="fdir", 
               stream_rast = "stream_r", 
               basins="basins", 
               flags="l", 
               overwrite = True)

# save
gs.run_command('r.out.gdal', input="basins", output= 'GIS South/Basins_Patagonia_all.tif', format='GTiff', overwrite=True)  

In [ ]:
# from raster to shp
data = rioxr.open_rasterio("GIS South/Basins_Patagonia_all.tif")
data.name = "gauge_id"
basins = vectorize(data)
basins = basins.set_crs(4326)
basins = basins.dissolve(by='gauge_id')
basins = basins.reset_index()

basins["total_area"]  = basins.to_crs(32719).area / 1e6
basins["lat"]         = basins.representative_point().y
basins["lon"]         = basins.representative_point().x

In [ ]:
# only study area
basins = basins[basins.lat  < -41.0]
basins = basins[basins.lon  < -68.0]
basins = basins[~((basins.lon > -71.4) & (basins.lat > -45.0) & (basins.lat < -40))]
basins = basins[~((basins.lon > -71.9) & (basins.lat > -41.3) & (basins.lat < -40.9))]
basins = basins[~((basins.lon > -72) & (basins.lat > -50.0) & (basins.lat < -44.7))]
basins = basins[~((basins.lon > -72.4) & (basins.lat > -49) & (basins.lat < -46))]
basins = basins[~((basins.lon > -72) & (basins.lat > -52.2) & (basins.lat < -50.4))]
basins = basins[~((basins.lon > -70.8) & (basins.lat > -51) & (basins.lat < -49))] 
basins = basins[~((basins.lon > -71) & (basins.lat > -52.4) & (basins.lat < -51))] 
basins = basins[~((basins.lon > -69.4) & (basins.lat > -54.2) & (basins.lat < -51))] 
basins = basins[~((basins.lon > -69.5) & (basins.lat > -54) & (basins.lat < -52))] 
basins = basins[~((basins.lon > -68.8) & (basins.lat > -54.5) & (basins.lat < -53))] 

# *small islands outside pmet coverage were manually removed in QGIS (n < 10)

# area greater than 4km2
basins = basins[basins.total_area > 4]

In [ ]:
# ID for each basin
names = np.linspace(start=1, stop=len(basins), num = len(basins), dtype = "int16").tolist()
names = [str(item).zfill(8) for item in names]
names = ["Y" + str(item) for item in names]
basins["gauge_id"] = names

# ID for each hydro zone
basins["ID_Zone"] = np.nan
basins.loc[basins.lat > -43.4, "ID_Zone"] = 1
basins.loc[(basins.lat < -43.4) & (basins.lat > -46), "ID_Zone"] = 2
basins.loc[(basins.lat < -46)   & (basins.lat > -47.8) & (basins.lon > -73), "ID_Zone"] = 3
basins.loc[(basins.lat < -46)   & (basins.lat > -47.8) & (basins.lon < -73), "ID_Zone"] = 4
basins.loc[(basins.lat < -47.8) & (basins.lat > -49.4), "ID_Zone"] = 5
basins.loc[(basins.lat < -49.4) & (basins.lat > -50.7), "ID_Zone"] = 6
basins.loc[(basins.lat < -50.7) & (basins.lat > -52.1), "ID_Zone"] = 7
basins.loc[(basins.lat < -52.1) & (basins.lat > -54.1), "ID_Zone"] = 8
basins.loc[basins.lat < -54.1, "ID_Zone"] = 9

# save file
basins.to_file("GIS South/Basins_Patagonia_all.shp")

## Delimitate all basins (outlet)

In [ ]:
# save outlet
basins = gpd.read_file("GIS South/Basins_Patagonia_all.shp")
basins = basins.set_index("gauge_id")
dem = rioxr.open_rasterio("GIS South/dem_patagonia3f_subset.tif")


In [ ]:
# old code in R
basins_xy = extract(dem, as.lines(all_basins), xy=TRUE)
basins_xy = basins_xy %>% group_by(ID) %>% slice_min(order_by = dem_patagonia3f)
basins_xy = aggregate(basins_xy, by = list(basins_xy$ID), FUN = mean) # several options 
basins_xy = basins_xy[,["ID", "x", "y"]]

basins_xy["gauge_id"] = all_basins.gauge_id 
basins_xy["area_km2"] = all_basins.total_area

basins_xy = vect(basins_xy, geom = c("x", "y"), crs="epsg:4326", keepgeom=T)
basins_xy.to_file("GIS South/Basins_Patagonia_all_mouth.shp")